# List of imports

In [3]:
from glob import glob
from pandas.core.common import flatten

# Creating datasets
Creating train, valid and test datasets. Train and valid sets are split 80/20.

In [21]:
train_data_path = 'data\\train'
test_data_path = 'data\\test'

classes, test_image_paths, train_image_paths = [], [], []

for data_path in glob(train_data_path + '\\*'):
    classes.append(" ".join(" - ".join(data_path.split('\\')[-1].split('___')).split('_')))
    train_image_paths.append(glob(data_path + '\\*'))

train_image_paths = list(flatten(train_image_paths))

for data_path in glob(test_data_path + '\\*'):
    test_image_paths.append(glob(data_path + '\\*'))

test_image_paths = list(flatten(test_image_paths))

split_index = int(len(train_image_paths) * 0.8)
train_image_paths, valid_image_paths = train_image_paths[:split_index], train_image_paths[split_index:]

print('class example: ', classes[0])
print('train_image_path example: ', train_image_paths[0])
print('test_image_path example: ', test_image_paths[0], end='\n\n')

print(f'Train size: {len(train_image_paths)}')
print(f'Valid size: {len(valid_image_paths)}')
print(f'Test size: {len(test_image_paths)}')


class example:  Apple - alternaria leaf spot
train_image_path example:  data\train\Apple___alternaria_leaf_spot\000413.jpg
test_image_path example:  data\test\Apple___healthy\1001.jpg.jpeg

Train size: 76374
Valid size: 19094
Test size: 2950
